In [292]:
import time
import math
import numpy as np
import scipy.io as sio
from numpy import linalg as la

In [293]:
#data i/o(read x_train, y_train, z_train, x_query, y_query,mu_pred)

data = sio.loadmat('whatIneed.mat')
x_train = data['x_gp']
u_train = data['u_gp']
z_train = data['z_gp']
y_train= np.concatenate((np.ones((n,1)), u_train), axis=1)
z_train = z_train.flatten()

n = len(x_train) #number of samples
d = len(x_train[0]) #dimension of the data
m = len(u_train[0]) #number of controls

x_test = x_train[0]
y_test = y_train[0]
z_test = z_train[0] #true value of the mu at query

In [294]:
#setting hyperparameters
rf_d = 50 #rf_d is dim of randomfeatures vector, to choose based on paper
mu = 0
sigma = 1
rf_mu = np.zeros(d)
rf_sigma = 1/sigma
rf_cov = rf_sigma * np.identity(d)
sigma_n = 1 #regularization parameter

In [295]:
#computing random features
#referring to random features as rf
#same sampling for different kernels

samples = np.random.multivariate_normal(rf_mu,rf_cov,size =rf_d//2) #(rf_d//2,d)

def rfquery_vector(x,y):    
    phi = np.empty(rf_d)
    dot_product = np.dot(samples, x)
    phi[0::2]=np.sin(dot_product)
    phi[1::2]=np.cos(dot_product)
    phi = math.sqrt(2/rf_d) * phi
    phis = y[:,np.newaxis] @ phi[np.newaxis,:]  #((m+1),d)
    return phis.flatten()
                         
def computeZ(x_train,y_train):
    phi = np.empty((n,rf_d))
    dot_product = x_train @ samples.T #(n,rf_d//2)
    phi[:,0::2]=np.sin(dot_product)
    phi[:,1::2]=np.cos(dot_product)
    phi = math.sqrt(2/rf_d) * phi
    Z = np.matmul(y_train[:,:,np.newaxis], phi[:,np.newaxis,:]).reshape((n,-1)) #(n,(m+1)*d)
    return Z

In [296]:
#computing compound kernel
#referring to compound as c
#using the same kernel

def compound_kernel(x_train,y_train):
    c_kernel = np.exp(-la.norm(x_train.reshape((n,1,d))-x_train, ord =2, axis=2)/(sigma ** 2))
    return (y_train@y_train.T) * c_kernel

def cquery_vector(x_train, x_test, y_train, y_test):
    c_vector = np.exp(-la.norm(x_train-x_test, ord =2, axis=1)/(sigma ** 2))
    return (y_train@y_test.T) * c_vector

In [297]:
# random features training
# w = (Z'Z+simga^2I)^{-1}Z'y
rf_start = time.time()
Z = computeZ(x_train, y_train) 
invZ_with_noise = la.inv(Z.T @Z + sigma_n ** 2 * np.identity((m+1)*rf_d))
rf_w = invZ_with_noise @ Z.T @ z_train #((m+1)*d,1)


#label prediction
phi_test = rfquery_vector(x_test,y_test)
rf_pred = np.dot(phi_test,rf_w)
rf_end = time.time()

In [298]:
# compound kernel training
# c_w = (K+simga^2I)^{-1}K(x)
ckernel_start = time.time()
c_kernel = compound_kernel(x_train,y_train)
c_vector = cquery_vector(x_train, x_test, y_train, y_test)
inv_ckernel_with_noise = la.inv(c_kernel + sigma_n**2 * np.identity(n))
c_w = inv_ckernel_with_noise @ c_vector


#label prediction
c_pred = np.dot(z_train,c_w)
ckernel_end = time.time()

In [299]:
np.set_printoptions(threshold=np.inf)
dif = c_kernel- Z @ Z.T < .001
print(dif.all())

False


In [300]:
#random features sigma computation
rfs_start = time.time()
rf_sigma = sigma**2 * phi_test.T @ invZ_with_noise @ phi_test
rfs_end = time.time()

#kernel sigma computation
ckernels_start = time.time()
kernel_sigma = compute_entry(x_test, x_test, y_test, y_test) -\
            c_vector.T @ inv_ckernel_with_noise @ c_vector
ckernels_end = time.time()

In [301]:
#print(f'{z_test=}{rf_pred=}{c_pred=}{rf_sigma=}{kernel_sigma=}') does not work for some reason
print(f'z_test={z_test} \nrf_pred={rf_pred} \nc_pred={c_pred}\n\
rf_sigma={rf_sigma} \nkernel_sigma={kernel_sigma}')
rf_mu_time = rf_end - rf_start
c_mu_time = ckernel_end - ckernel_start
print('rf_mu_time, c_mu_time', rf_mu_time, c_mu_time)

rf_sigma_time = rfs_end - rfs_start
c_sigma_time = ckernels_end - ckernels_start
print('rf_sigma_time, c_sigma_time', rf_sigma_time, c_sigma_time)

z_test=0.18107397972611733 
rf_pred=0.17078940673717183 
c_pred=0.18356798644315905
rf_sigma=0.01145902697613334 
kernel_sigma=0.12267424523537107
rf_mu_time, c_mu_time 0.011904716491699219 0.015638113021850586
rf_sigma_time, c_sigma_time 0.00026106834411621094 0.0011229515075683594


In [302]:
#change d    
#time constructing the matrix for socp 
#make figures?